In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

from fooof.sim.gen import gen_power_spectrum, gen_group_power_spectra, gen_periodic, gen_aperiodic
from fooof.sim.params import param_sampler, param_iter, param_jitter, Stepper
from fooof.plts.spectra import plot_spectrum, plot_spectra

from fooof import FOOOF, FOOOFGroup
from fooof.plts.fg import plot_fg
from fooof.plts.fg import plot_fg_peak_cens

In [3]:
def ap_gen(n_spectra):
    
    for spectrum in range(n_spectra):
        
        offset = np.float(np.random.uniform(0.75, 1, 1))
        exp = np.float(np.random.uniform (0.50, 1.50, 1))
        
        yield [offset, exp]
        
        
def p_gen(n_spectra):
    
    for n_spec in range(0, n_spectra, 10):
        
        # Increment CF in steps of 0.01
        delta_cf = 0.01 * n_spec
        
        # Increment BW between 0.25 and 1
        step = np.linspace(0.25, 1.0, 11)
        step_rev = np.linspace(0.25, 1.0, 11)[::-1]
        
        for sub_spec in range(n_spec, n_spec+10):

            CF1 = 10
            CF2 = 10 + delta_cf
           
            PW = 0.4
            
            BW1 = step[sub_spec-n_spec]
            BW2 = step_rev[sub_spec-n_spec]
        
        
            yield [[CF1, PW, BW1], [CF2, PW, BW2]]
        

# Comparison of CF versus CF+1SD Upper Bound
This notebook uses the new CF+1SD upper bound.

## Center Frequency
Both peaks start at 10 Hz and the second peak is increased by 0.1 Hz, every 10 models. The first peak is stationary at 10 Hz and the final second peak ends at 20 Hz, resulting in 1000 models. 

## Bandwidth
Within every set of 10 models, in which the CFs do not change, the bandwidths are set to:

(bandwidth_1, bandwidth_2)<br>

(0.25, 1.00)<br>
(0.32, 0.92)<br>
(0.40, 0.85)<br>
(0.48, 0.78)<br>
(0.55, 0.70)<br>
(0.62, 0.62)<br>
(0.70, 0.55)<br>
(0.78, 0.48)<br>
(0.85, 0.40)<br>
(0.92, 0.32)<br>
(1.00, 0.25)<br>

This is to be independent of which peak is first. 

## Gaussian Threshold
The same set of 1000 models, with varying CFs and BWs, are ran against 20 values for <code>_gauss_overlap_thresh</code>, ranging from 0.1 to 2.0 standard deviations.

## Other Parameters
PW is set to a constant 0.4 across all models.

Aperiodic parameters should be independent.

Noise level is set to 0.04.

In [4]:
np.random.seed(0)
n_spectra = 1000
freqs, powers = gen_group_power_spectra(n_spectra, [1, 35], ap_gen(n_spectra), p_gen(n_spectra), nlvs=0.04)

In [5]:
bad_peak_count = []
bad_sim_params = []
bw_mean = []
bw_std = []
cf_mean = []
cf_std = []

for thr in np.arange(0.1, 2.1, 0.1):
    
    fg = FOOOFGroup(peak_width_limits=(2, 11), min_peak_height=0.2)
    fg.add_data(freqs, powers)
    
    # Fit using a range of different threholds
    fg._gauss_overlap_thresh = thr
    fg.fit(n_jobs=8, progress='tqdm.notebook')
    
    # For reference of simulated params
    sim_params = list(p_gen(1000))
    
    # Count models with gt or lt 2 peaks
    peaks = fg.get_params('peak')
    bad_bw_diff = []
    bad_cf_diff = []
    for idx in range(n_spectra):
        peak_count = len(fg[idx].peak_params)
        if peak_count != 2:
            bad_bw_diff.append(np.diff(sim_params[idx], axis=0)[0][2])
            bad_cf_diff.append(np.diff(sim_params[idx], axis=0)[0][0])
            bad_sim_params.append(fg[idx].peak_params)
            
    bad_peak_count.append(len(bad_bw_diff))
                               
    bw_mean.append(np.mean(bad_bw_diff))
    bw_std.append(np.std(bad_bw_diff))
    cf_mean.append(np.mean(bad_cf_diff))
    cf_std.append(np.std(bad_cf_diff))

In [6]:
# Create a df of mean differences between two simulated peaks
#   for simulated pairs where gt or lt two peaks were fit
df_incorrect = pd.DataFrame()
df_incorrect["Threshold"] = np.arange(0.1, 2.1, 0.1)
df_incorrect['Peaks/1000'] = bad_peak_count
df_incorrect['Mean CF Diff'] = cf_mean   
df_incorrect['Std CF Diff'] = cf_std
df_incorrect['Mean BW Diff'] = bw_mean
df_incorrect['Std BW Diff'] = bw_std

In [7]:
df_incorrect

,Threshold,Peaks/1000,Mean CF Diff,Std CF Diff,Mean BW Diff,Std BW Diff
0,0.1,129,0.655814,0.681415,0.065116,0.420484
1,0.2,129,0.655814,0.681415,0.065116,0.420484
2,0.3,129,0.655814,0.681415,0.065116,0.420484
3,0.4,129,0.655814,0.681415,0.065116,0.420484
4,0.5,129,0.655814,0.681415,0.065116,0.420484
5,0.6,129,0.655814,0.681415,0.065116,0.420484
6,0.7,129,0.655814,0.681415,0.065116,0.420484
7,0.8,151,0.729801,0.475742,0.049669,0.416366
8,0.9,157,0.775796,0.522537,0.047771,0.419180
9,1.0,157,0.775796,0.522537,0.047771,0.419180


In [12]:
# Compare to orignal threshold, where CF was used as upper bounds
df_orig = pd.DataFrame()
df_orig['Threshold'] = np.arange(0.1, 2.1, 0.1)
df_orig['Peaks/1000'] = [129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 154, 154, 154, 154, 157]
df_orig['Mean CF Diff'] = [0.655814, 0.655814, 0.655814, 0.655814, 0.655814, 0.655814, 0.655814, 0.655814, 0.655814, 0.655814, 0.655814, 0.655814, 0.655814, 0.655814, 0.655814, 0.757143, 0.757143, 0.757143, 0.757143, 0.787898]  
df_orig['Std CF Diff'] = [0.681415, 0.681415, 0.681415, 0.681415, 0.681415, 0.681415, 0.681415, 0.681415, 0.681415, 0.681415, 0.681415, 0.681415, 0.681415, 0.681415, 0.681415, 0.509502, 0.509502, 0.509502, 0.509502, 0.551356
]
df_orig['Mean BW Diff'] = [0.065116, 0.065116, 0.065116, 0.065116, 0.065116, 0.065116, 0.065116, 0.065116, 0.065116, 0.065116, 0.065116, 0.065116, 0.065116, 0.065116, 0.065116, 0.057468, 0.057468, 0.057468, 0.057468, 0.069745
]
df_orig['Std BW Diff'] = [0.420484, 0.420484, 0.420484, 0.420484, 0.420484, 0.420484, 0.420484, 0.420484, 0.420484, 0.420484, 0.420484, 0.420484, 0.420484, 0.420484, 0.420484, 0.417039, 0.417039, 0.417039, 0.417039, 0.422412]
df_orig

,Threshold,Peaks/1000,Mean CF Diff,Std CF Diff,Mean BW Diff,Std BW Diff
0,0.1,129,0.655814,0.681415,0.065116,0.420484
1,0.2,129,0.655814,0.681415,0.065116,0.420484
2,0.3,129,0.655814,0.681415,0.065116,0.420484
3,0.4,129,0.655814,0.681415,0.065116,0.420484
4,0.5,129,0.655814,0.681415,0.065116,0.420484
5,0.6,129,0.655814,0.681415,0.065116,0.420484
6,0.7,129,0.655814,0.681415,0.065116,0.420484
7,0.8,129,0.655814,0.681415,0.065116,0.420484
8,0.9,129,0.655814,0.681415,0.065116,0.420484
9,1.0,129,0.655814,0.681415,0.065116,0.420484


## Results

The original threshold needs to be lowered since peaks were originally removed if:<br>
*guess_freq_0 > guess_freq_1 - (guess_std_1 * _gauss_overlap_thresh)*

The new conditional to drop a peak has become:<br>
*guess_freq_0 + __(guess_std_0 * _gauss_overlap_thresh)__ > guess_freq_1 - (guess_std_1 * _gauss_overlap_thresh)*

This means the same <code>_gauss_overlap_thresh</code> of 1.5 will allow this condition to be more easily met.

This new conditional can be re-arranged:<br>
*_gauss_overlap_thresh * (guess_std_0 + guess_std_1) > guess_freq_1 - guess_freq_0*

And since guess_std_0 was zero originally, the new to old threshold is:<br>
new *_gauss_overlap_thresh * (guess_std_0 + guess_std_1) =* orig *_gauss_overlap_thresh * guess_std_1*

And if guess_std_0 ~= guess_std_1, then:<br>
__new__ *_gauss_overlap_thresh * 2 guess_std =* __orig__*_gauss_overlap_thresh * guess_std* <br>
__new__ *_gauss_overlap_thresh =* __orig__*_gauss_overlap_thresh / 2*

In other words, in cases where the standard deviation guess of the two gaussians are equal, the new threshold should be <code>0.75</code>. If the two standard deviations are not equal, the new threshold is more complicated to maintain the original <code>1.5</code> cutoff:<br>
__new__ *_gauss_overlap_thresh =* __orig__*_gauss_overlap_thresh * guess_std+1 / (guess_std_0 + guess_std_1)*

A threshold of 0.75 is also found experimentally above.